In [1]:
import torch
import numpy as np
import random

In [2]:
seed = 42
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
random.seed(seed)

# Define Model

In [3]:
from utils.DevConf import DevConf
DEV_CONF = DevConf(device='cuda')

In [4]:
from utils.AttnBlocksConf import AttnBlocksConf
from model.BertDecoder.SentiClassifier import SentiClassifier
from model.CombinationModel import CombinationModel
from utils.const import BlockType

c:\Users\henry\.conda\envs\torch-env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [59]:
mapper = SentiClassifier(6, AttnBlocksConf(768, 12, nKVHead=6), BlockType.CROSS, devConf=DEV_CONF)
model = CombinationModel(nClass=6, decoder=mapper, devConf=DEV_CONF)

# Load Data

In [6]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-multilingual-cased", cache_dir='./cache/tokenizer')

In [7]:
import pandas as pd

In [8]:
train = pd.read_csv('data/archive/train.csv')

In [9]:
import torch
from torch.utils.data import Dataset, DataLoader, random_split

In [10]:
class MyDataset(Dataset):
    def __init__(self, df, tokenizer):
        self.df = df
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        # print(idx)
        text = self.df.iloc[idx]['ABSTRACT']
        label = torch.tensor([self.df.iloc[idx][i] for i in ["Computer Science","Physics","Mathematics","Statistics","Quantitative Biology","Quantitative Finance"]])
        return text, label
        # return self.tokenizer(text, return_tensors='pt', padding='max_length', truncation=True, max_length=128), label

In [11]:
def collect_fn(batch):
    texts, labels = zip(*batch)
    # print(texts)
    return tokenizer(texts, return_tensors='pt', padding='max_length', truncation=True, max_length=512).to(device=DEV_CONF.device), torch.stack(labels).to(DEV_CONF.device)

In [12]:
# def seed_worker(worker_id):
#     worker_seed = torch.initial_seed() % 2**32
#     np.random.seed(worker_seed)
#     random.seed(worker_seed)

g = torch.Generator()
g.manual_seed(0)

In [13]:
dataset = MyDataset(train, tokenizer)

datasize = len(dataset)
splitIndex = int(datasize * 0.2)
trainDataSize = datasize - splitIndex

train_dataset, test_dataset = random_split(dataset, [trainDataSize, splitIndex])

train_loader = DataLoader(
    train_dataset, collate_fn=collect_fn, batch_size=8, shuffle=True,
    generator=g)
test_loader = DataLoader(
    test_dataset, collate_fn=collect_fn, batch_size=1, shuffle=True,
    generator=g)

In [30]:
print(trainDataSize, splitIndex)

16778 4194


# Train

In [14]:
from torch import nn
from torch.utils.tensorboard import SummaryWriter

In [60]:
lr = 1e-5
epochs = 1
loss_fn = nn.BCELoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)

In [43]:
def train_fn(model, train_loader, loss_fn, optimizer, epochs):
    model.train()
    writer = SummaryWriter()
    for epoch in range(epochs):
        for i, (data, label) in enumerate(train_loader):
            # print(data['input_ids'])
            # break
            optimizer.zero_grad()
            output = model(**data)#, NoGradBert=False)
            loss = loss_fn(output, label.float())
            loss.backward()
            optimizer.step()
            if i % 100 == 99:
                print(f"Epoch {epoch+1}/{epochs} - Batch {i+1}/{len(train_loader)} - Loss: {loss.item()}")
                writer.add_scalar('Loss/train', loss.item(), i + 1)
    writer.flush()
    writer.close()


In [61]:
model.train()
train_fn(model, train_loader, loss_fn, optimizer, epochs)
print("Done")

Epoch 1/1 - Batch 100/2098 - Loss: 0.5620404481887817
Epoch 1/1 - Batch 200/2098 - Loss: 0.38083696365356445
Epoch 1/1 - Batch 300/2098 - Loss: 0.2401605248451233
Epoch 1/1 - Batch 400/2098 - Loss: 0.30665573477745056
Epoch 1/1 - Batch 500/2098 - Loss: 0.26897913217544556
Epoch 1/1 - Batch 600/2098 - Loss: 0.22492913901805878
Epoch 1/1 - Batch 700/2098 - Loss: 0.47321945428848267
Epoch 1/1 - Batch 800/2098 - Loss: 0.1846269965171814
Epoch 1/1 - Batch 900/2098 - Loss: 0.28365057706832886
Epoch 1/1 - Batch 1000/2098 - Loss: 0.3762596547603607
Epoch 1/1 - Batch 1100/2098 - Loss: 0.4708443880081177
Epoch 1/1 - Batch 1200/2098 - Loss: 0.12179668992757797
Epoch 1/1 - Batch 1300/2098 - Loss: 0.1721089780330658
Epoch 1/1 - Batch 1400/2098 - Loss: 0.11529495567083359
Epoch 1/1 - Batch 1500/2098 - Loss: 0.15808045864105225
Epoch 1/1 - Batch 1600/2098 - Loss: 0.18334411084651947
Epoch 1/1 - Batch 1700/2098 - Loss: 0.23328182101249695
Epoch 1/1 - Batch 1800/2098 - Loss: 0.11077529191970825
Epoch 1

# Eval

In [62]:
model.eval()

CombinationModel(
  (distilBert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (lin1):

In [19]:
# id = 13
# sample = tokenizer(dataset[id][0], return_tensors='pt', padding='max_length', truncation=True, max_length=512).to(device=DEV_CONF.device)
# output = torch.where(model(**sample) > 0.5, 1, 0)
# print(output)
# print(dataset[id][1])
# array = output.to("cpu").squeeze().numpy()
# dataset_array = dataset[id][1].numpy()
# for i in range(6):
#     print(array[i], dataset_array[i])
# ans = torch.eq(output.to("cpu"), dataset[id][1])
# print(ans)
# print(torch.all(ans))

In [20]:
# acc = [0] * 6
# testdata = train.sample(1000)
# test_dataset = MyDataset(testdata, tokenizer)

# for data in test_dataset:
#     # print(data[1][0])
#     sample = tokenizer(data[0], return_tensors='pt', padding='max_length', truncation=True, max_length=512).to(device=DEV_CONF.device)
#     output = torch.where(model(**sample) > 0.5, 1, 0)
#     ansList = torch.eq(output.squeeze().to("cpu"), data[1])
#     for i, ans in enumerate(ansList):
#         # print(ans)
#         if ans:
#             acc[i] += 1

In [21]:
# print(acc)

In [77]:
def test(model, test_loader):
    acc = [[[0, 0], [0, 0]] for _ in range(6)]
    for (data, label) in test_loader:
        output = torch.where(model(**data) > 0.25, 1, 0).squeeze().cpu().numpy()
        ans = label.squeeze().squeeze().cpu().numpy()
        for i in range(6):
            acc[i][output[i]][ans[i]] += 1
    return acc

In [78]:
acc = test(model, test_loader)

In [79]:
for matrix in acc:
    print(matrix)

[[1816, 84], [677, 1617]]
[[2795, 179], [223, 997]]
[[2839, 274], [197, 884]]
[[2855, 326], [259, 754]]
[[4060, 87], [14, 33]]
[[4154, 40], [0, 0]]


[[1606, 887], [66, 1635]]
[[2565, 453], [154, 1022]]
[[2581, 455], [179, 979]]
[[1836, 1278], [112, 968]]
[[4074, 0], [120, 0]]
[[4154, 0], [40, 0]]

[[1513, 980], [45, 1656]]
[[2438, 580], [114, 1062]]
[[2727, 309], [227, 931]]
[[2112, 1002], [155, 925]]
[[4074, 0], [120, 0]]
[[4154, 0], [40, 0]]

[[1885, 608], [102, 1599]]
[[2640, 378], [111, 1065]]
[[2581, 455], [143, 1015]]
[[2388, 726], [135, 945]]
[[4073, 1], [111, 9]]
[[4154, 0], [40, 0]]

[[1893, 600], [63, 1638]]
[[2732, 286], [99, 1077]]
[[2646, 390], [127, 1031]]
[[2474, 640], [59, 1021]]
[[4007, 67], [41, 79]]
[[4116, 38], [12, 28]]

[[1850, 643], [59, 1642]]
[[2556, 462], [54, 1122]]
[[2667, 369], [136, 1022]]
[[2804, 310], [180, 900]]
[[4018, 56], [44, 76]]
[[4145, 9], [27, 13]]

[[1870, 623], [108, 1593]]
[[2322, 696], [83, 1093]]
[[2450, 586], [107, 1051]]
[[2403, 711], [123, 957]]
[[4038, 36], [69, 51]]
[[4150, 4], [25, 15]]

[[1585, 908], [38, 1663]]
[[2578, 440], [111, 1065]]
[[2755, 281], [209, 949]]
[[2377, 737], [93, 987]]
[[4035, 39], [60, 60]]
[[4154, 0], [40, 0]]

[[1656, 837], [45, 1656]]
[[2680, 338], [144, 1032]]
[[2761, 275], [203, 955]]
[[2674, 440], [223, 857]]
[[4048, 26], [73, 47]]
[[4153, 1], [34, 6]]

In [80]:
microacc = [[0, 0], [0, 0]]
for i in range(6):
    for j in range(2):
        for k in range(2):
            microacc[j][k] += acc[i][j][k]

In [81]:
print(microacc) # microaveraging, macroaveraging

[[18519, 990], [1370, 4285]]


[[16816, 3073], [671, 4604]]

[[17018, 2871], [701, 4574]]

[[17721, 2168], [642, 4633]]

[[17868, 2021], [401, 4874]]

[[18040, 1849], [500, 4775]]

[[17233, 2656], [515, 4760]]

[[17484, 2405], [551, 4724]]

[[17972, 1917], [722, 4553]]

In [82]:
recall = microacc[1][1] / (microacc[1][1] + microacc[1][0])
precision = microacc[1][1] / (microacc[1][1] + microacc[0][1])
f1 = 2 * recall * precision / (recall + precision)
acc = (microacc[0][0] + microacc[1][1]) / (microacc[0][0] + microacc[1][1] + microacc[0][1] + microacc[1][0])
print(f"Recall: {recall}\nPrecision: {precision}\nF1: {f1}\nAcc: {acc}")

Recall: 0.757736516357206
Precision: 0.8123222748815165
F1: 0.7840805123513266
Acc: 0.9062152281036401


Recall: 0.8854976303317535
Precision: 0.6786285050123493
F1: 0.7683829577233097
Acc: 0.8880941026863773

Recall: 0.8671090047393365
Precision: 0.6143720617864339
F1: 0.7191823899371069
Acc: 0.8580511842314418

Recall: 0.8782938388625593
Precision: 0.6812233495074254
F1: 0.7673070553163299
Acc: 0.8883325385471308

Recall: 0.9239810426540285
Precision: 0.7068890500362581
F1: 0.8009860312243221
Acc: 0.9037513908758544

Recall: 0.9052132701421801
Precision: 0.7208635265700483
F1: 0.8025884528111605
Acc: 0.9066523605150214

Recall: 0.9023696682464455
Precision: 0.6418554476806904
F1: 0.7501378929950357
Acc: 0.8739866475917978

Recall: 0.8955450236966824
Precision: 0.6626455323327255
F1: 0.7616897774911319
Acc: 0.8825305992687967

Recall: 0.8631279620853081
Precision: 0.70370942812983
F1: 0.7753086419753087
Acc: 0.8951279605786043

In [83]:
# torch.save(model.state_dict(), 'weights/model-240520-2206-Cross-NoGrad-epoch1.pth')

In [84]:
# model.load_state_dict(torch.load('weights/model-240520-epoch1.pth'))